# Setup

In [ ]:
!apt-get update
!apt-get install git

#!pip uninstall scikit-learn
!pip install scikit-learn===0.24.0 #needed for autopytorch but need >=1.0 needed for handcrafted feature generation or get https://github.com/mne-tools/mne-features/issues/73
#!pip install scikit-learn

# MNE-BIDS preprocessing functions
!pip install mne
!pip install MNE-bids
#!pip install -r https://raw.githubusercontent.com/mne-tools/mne-bids-pipeline/main/requirements.txt
!pip install typing_extensions psutil joblib dask[distributed] jupyter-server-proxy scikit-learn pandas seaborn json_tricks coloredlogs python-picard fire pyqt5 pyvista pyvistaqt openpyxl

!git clone https://github.com/mne-tools/mne-bids-pipeline.git /cloned-mne-bids-pipeline

!pip install AutoReject
!pip install coffeine
!pip install Braindecode

#!rm -r /cloned-paper-repo
!git clone https://github.com/meeg-ml-benchmarks/brain-age-benchmark-paper.git /cloned-paper-repo

!pip install -U matplotlib

!pip install -U scipy
!pip install torch torchvision -f https://download.pytorch.org/whl/torch_stable.html

!pip install numpy matplotlib scipy numba scikit-learn mne PyWavelets pandas


### Restart Runtime 

make sure all updated packages are loaded in their newest versions


In [ ]:
import os
def restart_runtime():
  os.kill(os.getpid(), 9)
restart_runtime()

###Mount Google Drive for access to data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
######################################################################################################
#  ________   _________   ___    ___ ___  ___  ________  ________  ___  __      _______   _______
# |\   ___  \|\___   ___\|\  \  /  /|\  \|\  \|\   __  \|\   ____\|\  \|\  \   /  ___  \ /  ___  \
# \ \  \\ \  \|___ \  \_|\ \  \/  / | \  \\\  \ \  \|\  \ \  \___|\ \  \/  /|_/__/|_/  //__/|_/  /|
#  \ \  \\ \  \   \ \  \  \ \    / / \ \   __  \ \   __  \ \  \    \ \   ___  \__|//  / /__|//  / /
#   \ \  \\ \  \   \ \  \  /     \/   \ \  \ \  \ \  \ \  \ \  \____\ \  \\ \  \  /  /_/__  /  /_/__
#    \ \__\\ \__\   \ \__\/  /\   \    \ \__\ \__\ \__\ \__\ \_______\ \__\\ \__\|\________\\________\
#     \|__| \|__|    \|__/__/ /\ __\    \|__|\|__|\|__|\|__|\|_______|\|__| \|__| \|_______|\|_______|
#                        |__|/ \|__|
######################################################################################################

import mne
import numpy as np
import pandas as pd
import sklearn
from sklearn.pipeline import make_pipeline
from mne.decoding import Vectorizer
from sklearn.decomposition import PCA
from sklearn.linear_model import Ridge

# Merge handcrafted, fooofs, fbriemann

In [ ]:
####################### load handcrafteds ##############################################
import pickle
%cd /content/drive/My Drive/brainAge_competition/
filename = 'trainData_250hz_2secEpochs_handcraftedFeatures.sav' 
with open(filename, "rb") as input_file:
  trainSet_Epochs_averaged2 = pickle.load(input_file)

  
##################### fooofs data ##############################################

import pickle
%cd /content/drive/My Drive/brainAge_competition/
filename = 'fooofResults' 
with open(filename, "rb") as input_file:
  fooofResults = pickle.load(input_file)
print(fooofResults.shape) #1782
type(fooofResults)
fooofResults = np.array(fooofResults)
fooofResults

######################## load filterbank riemann ##############################################
import pickle
%cd /content/drive/My Drive/brainAge_competition/
with open(r"trainData_250hz_2secEpochs_filterbankRiemannFeatures.sav", "rb") as input_file:
  fbriemannFeats_train = pickle.load(input_file)

covs = [sub['covs'] for sub in fbriemannFeats_train]
covs = np.array(covs)

covs2 = [None] * 1200
for i in range(0,1200,1):
  covs2[i]=covs[i].flatten()
len(covs2)
covs3 = np.array(covs2) 

######################## age predictions from best fb riemann model ##############################################
import pickle
%cd /content/drive/My Drive/brainAge_competition/

with open(r"trainData_filterbankRiemann_ridgeCV_predictions.sav", "rb") as input_file:
  fbriemann_trainPreds = pickle.load(input_file)

with open(r"validData_filterbankRiemann_ridgeCV_predictions.sav", "rb") as input_file:
  fbriemann_validPreds = pickle.load(input_file)

fbriemann_Preds = np.concatenate((fbriemann_trainPreds,fbriemann_validPreds))

######################## predictions from DSF attempt2   ##############################################

with open(r"trainSet_DSFattempt2_predictions.sav", "rb") as input_file:
  per_ptp_avg_y_pred_trainSet = pickle.load(input_file)

###################################################### merge ######################################################
trainhandcrafted_and_fooof = np.c_[trainSet_Epochs_averaged2, trainfooof_params_toMergeWithHandcrafted]
trainhandcrafted_and_fooof2 =  np.c_[trainhandcrafted_and_fooof, fooofResults]
trainhandcrafted_fooof_fbriemann = np.c_[trainhandcrafted_and_fooof2, covs3]
trainhandcrafted_fooof_fbriemann_fbrAgePreds = np.c_[trainhandcrafted_fooof_fbriemann, fbriemann_Preds]
trainhandcrafted_fooof_fbriemann_fbrAgePreds_deep4Preds = np.c_[trainhandcrafted_fooof_fbriemann_fbrAgePreds, per_ptp_avg_y_pred_trainSet]


#  prepare X and y

In [ ]:
########### X Train/Valid Split ###################

X_trainSet_preScaled = trainhandcrafted_fooof_fbriemann_fbrAgePreds_deep4Preds[:]
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_trainSet = scaler.fit_transform(X_trainSet_preScaled)
print(X_trainSet.shape)

############### train ys #####################
train_path = "/content/drive/My Drive/brainAge_competition/raws/"
train_subj = 1200
meta = pd.read_csv(train_path + "train_subjects.csv")
y_train = []
for age in meta["age"][:train_subj]:
    y_train.append(age)

y_trainSet = y_train


# torch


In [ ]:
!pip install torch
import torch
torch.cuda.is_available()
torch.cuda.get_device_name(0)
!pip install autoPyTorch
import autoPyTorch
from autoPyTorch.api.tabular_regression import TabularRegressionTask
from autoPyTorch.datasets.resampling_strategy import CrossValTypes

import os
import tempfile as tmp
import warnings

import sklearn.datasets
import sklearn.model_selection

os.environ['JOBLIB_TEMP_FOLDER'] = tmp.gettempdir()
os.environ['OMP_NUM_THREADS'] = '1'
os.environ['OPENBLAS_NUM_THREADS'] = '1'
os.environ['MKL_NUM_THREADS'] = '1'

warnings.simplefilter(action='ignore', category=UserWarning)
warnings.simplefilter(action='ignore', category=FutureWarning)

import multiprocessing
multiprocessing.cpu_count()

import gc
gc.collect()
torch.cuda.empty_cache()

api = TabularRegressionTask(seed=42, 
                            n_jobs=4, 
                            n_threads=2,
                            resampling_strategy=CrossValTypes.k_fold_cross_validation,
                            precision = 16,
                            temporary_directory = "/content/drive/My Drive/brainAge_competition/temp5v4",
                            output_directory = "/content/drive/My Drive/brainAge_competition/temp_output5v4",
                            delete_tmp_folder_after_terminate =False,
                            )

api.search(
    X_train=X_trainSet,
    y_train=y_trainSet,
    #X_test=X_testSet,
    #y_test=y_validSet,
    dataset_name='foobar',
    optimize_metric='mean_absolute_error', 
    total_walltime_limit=(2*60*60), 
    func_eval_time_limit_secs=(2*60*60)/2, 
    memory_limit=4000, 
    precision = 16
)
print(api.sprint_statistics())

import pickle
%cd /content/drive/My Drive/brainAge_competition/potentialSubmissions/
filename = 'potentialSubmission_Analysis5v4_mdl.sav' 
pickle.dump(api, open(filename, 'wb'))

test_predictions = api.predict(X_testSet)
test_predictions
for i in range(0,400,1):
  print(test_predictions[i])

import pickle
%cd /content/drive/My Drive/brainAge_competition/potentialSubmissions/
filename = 'potentialSubmission_Analysis5v4_testPreds.sav' 
pickle.dump(test_predictions, open(filename, 'wb'))

